In [10]:
import numpy as np
import tensorflow as tf
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib
import matplotlib.pyplot as plt

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01
l2_reg = 0.0001

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
my_dense_layer = partial(tf.layers.dense, 
                         activation=tf.nn.elu, 
                         kernel_initializer=he_init,
                         kernel_regularizer=l2_regularizer)

hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3 = my_dense_layer(hidden2, n_hidden3)
outputs = my_dense_layer(hidden3, n_outputs, activation=None)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimzer = tf.train.AdamOptimizer(learning_rate)
training_op = optimzer.minimize(loss)

init = tf.global_variables_initializer()

mnist = input_data.read_data_sets("/tmp/data/")
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels
n_test_digits = 5
X_test = mnist.test.images[:n_test_digits]

n_epochs = 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_train = loss.eval(feed_dict={X: X_batch})
        loss_test = loss.eval(feed_dict={X: X_test})
        print(epoch, "Train loss:", loss_train, "Test loss:", loss_test)
    outputs_val = outputs.eval(feed_dict={X: X_test})

plt.figure()
for digit_index in range(n_test_digits):
    plt.subplot(n_test_digits, 2, digit_index * 2 + 1)
    plt.imshow(X_test[digit_index].reshape([28, 28]), cmap="Greys", interpolation="nearest")
    plt.axis("off")
    plt.subplot(n_test_digits, 2, digit_index * 2 + 2)
    plt.imshow(outputs_val[digit_index].reshape([28, 28]), cmap="Greys", interpolation="nearest")
    plt.axis("off")
plt.savefig("../plots/ex_15_02.pdf")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train loss: 0.0854492 Test loss: 0.0808164
1 Train loss: 0.0380459 Test loss: 0.0361185
2 Train loss: 0.026848 Test loss: 0.0255562
3 Train loss: 0.0236398 Test loss: 0.0223829
4 Train loss: 0.0218107 Test loss: 0.0218849
5 Train loss: 0.0229394 Test loss: 0.0216914
6 Train loss: 0.0228827 Test loss: 0.0216425
7 Train loss: 0.0236355 Test loss: 0.0222498
8 Train loss: 0.0250524 Test loss: 0.0234064
9 Train loss: 0.0230265 Test loss: 0.0219656
